In [1]:
DATASET_NAME = "Deco"

DATA_DIR = "data"
REFINED_PATH = f"../output/{DATASET_NAME}/refined.json"
PAPER_RESULTS = f"../output/{DATASET_NAME}/NoTrainingNoSeed"
OWN_RESULTS = f"../output/{DATASET_NAME}/OWN_NoTrainingNoSeed"

In [2]:
import json
from os.path import join

from experiments.Analyser import *
from labelregions.BoundingBox import BoundingBox

with open(REFINED_PATH) as f:
    refined = json.load(f)


def paper_results(key):
    with open(join(PAPER_RESULTS, f"{key}_result.json")) as f:
        data = json.load(f)
    return result_eval(key, data)


def own_results(key):
    with open(join(OWN_RESULTS, f"{key}_result.json")) as f:
        data = json.load(f)
    return result_eval(key, data)
    
    
def result_eval(key, data):
    ground_truth = [BoundingBox(r["top"], r["left"], r["bottom"], r["right"]) for r in data["ground_truth"]]
    detected = [BoundingBox(r["top"], r["left"], r["bottom"], r["right"]) for r in data["detected"]]
    
    
    results = {
        **bayesian_scores(ground_truth, detected),
        **detection_evaluation(ground_truth, detected),
        "accuracy": accuracy_based_on_jacard_index(ground_truth, detected)
    }
    return results


In [3]:
from os import listdir

keys = listdir(OWN_RESULTS)

In [4]:
file_diffs = {}
metric_diffs = {}

for file in refined.keys():
    own = own_results(file)
    paper = paper_results(file)
    
    diffs = {}
    for key in own.keys():
        diff = own[key] - paper[key]

        diffs[key] = diff
        metric_diffs[key] = metric_diffs.get(key, []) + [diff]
    file_diffs[file] = diffs